In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import concurrent.futures
from sklearn.model_selection import KFold
from glmpy.models import ZINBModel
from scipy.stats import nbinom, bernoulli

# Function to generate data
def generate_data(size, pi=0.3):
    cell_types = ["Cardiomyocytes", "Ectoderm", "Pluripotent"]
    probabilities = [0.4, 0.2, 0.4]
    mu_values = [13, 50, 200]
    alpha_values = [5, 10, 20]  # Dispersion parameters for each cell type

    cell_type_samples = np.random.choice(cell_types, size=size, p=probabilities)
    y_samples = np.zeros(size, dtype=int)
    
    for i, cell_type in enumerate(cell_types):
        indices = np.where(cell_type_samples == cell_type)[0]
        mu = mu_values[i]
        alpha = alpha_values[i]
        r = alpha
        p = alpha / (mu + alpha)
        zero_inflated = bernoulli.rvs(pi, size=len(indices))
        y_samples[indices] = np.where(zero_inflated == 1, 0, nbinom.rvs(r, p, size=len(indices)))
    
    covariates = {
        "cell_type": cell_type_samples,
    }
    
    return y_samples, covariates

# Custom scoring function
def score_model(params, y_train, covariates_train, y_test, covariates_test):
    model = ZINBModel(y=y_train, covariates=covariates_train, name="ZIN")
    model.fit(
        optimization_method="dogleg",
        verbose=False,
        parallel=False,
        ret_time=False,
        regularization_params=params,
        safe=False,
        minimize_options={"disp": False, "maxiter": None},
    )
    return -model.loglik(model.best_params)  # Negative log-likelihood

# Function to perform grid search
def grid_search(y, covariates, param_grid, cv_splits=5):
    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)
    
    results = []
    with concurrent.futures.ProcessPoolExecutor(6) as executor:
        future_to_params = {}
        
        for params in param_grid:
            for train_index, test_index in kf.split(y):
                y_train, y_test = y[train_index], y[test_index]
                covariates_train = {k: np.array(v)[train_index] for k, v in covariates.items()}
                covariates_test = {k: np.array(v)[test_index] for k, v in covariates.items()}
                
                future = executor.submit(score_model, params, y_train, covariates_train, y_test, covariates_test)
                future_to_params[future] = params

        for future in concurrent.futures.as_completed(future_to_params):
            params = future_to_params[future]
            try:
                score = future.result()
                results.append((params, score))
            except Exception as e:
                print(f"Exception occurred for parameters {params}: {e}")
    
    best_params, best_score = min(results, key=lambda x: x[1])
    return best_params, best_score

# Generate data
y, covariates = generate_data(size=10000)

# Define the parameter grid
lambdas = np.logspace(-9, -3, num=7)  # Generating values from 1e-9 to 1e-3
constants = np.logspace(-9, -3, num=7)  # Same range for constants
thresholds = np.logspace(-10, -7, num=4)  # Generating values from 1e-10 to 1e-7

param_grid = []

# Ridge and Norm methods
for l in lambdas:
    param_grid.append({'method': 'ridge', 'lambda': l})
    param_grid.append({'method': 'norm', 'lambda': l})

# Norm_constant method
for l in lambdas:
    for c in constants:
        param_grid.append({'method': 'norm_constant', 'lambda': l, 'constant': c})

# Eigenvalue method
for t in thresholds:
    param_grid.append({'method': 'eigenvalue', 'threshold': t})

# Perform grid search
best_params, best_score = grid_search(y, covariates, param_grid)

print("Best parameters found: ", best_params)
print("Best cross-validation score: ", best_score)


/home/rg972/.conda/envs/poop/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:404: UserWarning: [<class 'dict'>] is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/home/rg972/.conda/envs/poop/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:404: UserWarning: [<class 'dict'>] is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
ic| ic|hessian_matrix: ic|hessian_matrix tensor:( [ [tensorichessian_matrix :|(0.0000e+00 ,[ hessian_matrixtensor[   :0.0000e+000.0000e+00,   tensor0.0000e+00(,,  [0.0000e+00,  [   0.0000e+000.0000e+00,, 

Exception occurred for parameters {'method': 'ridge', 'lambda': 9.999999999999999e-10}: sigmoid(): argument 'input' (position 1) must be Tensor, not numpy.float64


   0.0000e+00,,0.0000     0.0000,   0.00001.2922e+01,]]   , ,
                           2.9106dtype-=13.5773,torch,   0.0000  ,- -0.2258,21.2098   .,float640.0000
,)                                  0.0000
0.0000,,   0.0000
]                               0.0000,  15.4287]], ,
                            [dtype=torch  .0.0000,   0.0000float64, )110.9541,   0.0000,   1.6113,   0.0000
,   0.0000,
                             -12.2690,   0.0000],
                            [  0.0000,   0.0000,   0.0000, 210.6940,   1.7616,   0.0000,   0.0000,
                               0.0000, -13.6805],
                            [  0.0000,   ic2.9106|,   1.6113,   1.7616,   10.0239,   hessian_matrix: tensor([[0.0000,  -1.2021,
                              - 0.3105,  0.0000e+00,-0.2115]  ,
0.0000e+00                            [,  0.0000e+00  0.0000,  ,0.0000e+00,   0.0000  ,0.0000e+00,   
0.0000                              ,0.0000e+00   ,  0.0000e+00,  0.0000,0.0000e+00,     0.0000e+00],
0.0000

Exception occurred for parameters {'method': 'ridge', 'lambda': 9.999999999999999e-10}: sigmoid(): argument 'input' (position 1) must be Tensor, not numpy.float64



                            [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -5.1117e+01, -1.6643e-02,
                              0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0293e+01]],
                           dtype=torch.float64)
ic| hessian_matrix: tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
                               0.0000,   0.0000],
                            [  0.0000,  77.1368,   0.0000,   0.0000,   2.6728,   0.0000, -21.4688,
                               0.0000,   0.0000ic]|, 
                            hessian_matrix[  : tensor([[0.0000,     0.00000.0000,,   0.0000,   0.0000,   0.0000,    0.0000,ic   108.67640.0000,   0.0000|ic,,   1.6419   |, 0.0000hessian_matrix,:
                                0.0000,tensor    hessian_matrix0.0000:    0.0000(tensor[],,([[  
                               [0.0000,[0.0000
                             -   ,0.0000e+0012.37700.0000   ,,,0.0000       ,0.00000.0000e+00],  76.42960.0000e+00   0.0000,   0.0000

...

,
                              0.0000e+00,  0.0000e+00, -1.2814e+01,  ,  0.0000e+00,  7.3417e+00,  0.0000e+00],
                            [ 0.0000e+00,  0.0000e+00,  0.0000e+00]0.0000e+00, -,2.7022e+01
,                             [ -0.0000e+00,3.0171e-01,
                                0.0000e+000.0000e+00,,    0.0000e+00,  0.0000e+000.0000e+00,,    1.0841e+015.4232e+02],]  2.8299e+00,,
                              
                           dtype0.0000e+00,=  0.0000e+00torch.,float64)  0.0000e+00ic,
|  hod- =2.7365e+01] ,
                            regulari:[  0.0000e+00tensor,  2.8794e+00,  1.6801e+00,  ([2.8299e+00[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,,     1.1984e+010.0000,
,
                               0.0000                              ,   0.0000],
                            [  0.0000,  66.56020.0000e+00,,    0.0000-1.2390e+00, -3.6272e-01,   ,0.0000 -2.6729e-01],
                            [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, 

Exception occurred for parameters {'method': 'norm', 'lambda': 1e-08}: sigmoid(): argument 'input' (position 1) must be Tensor, not numpy.float64
Exception occurred for parameters {'method': 'norm', 'lambda': 1e-08}: sigmoid(): argument 'input' (position 1) must be Tensor, not numpy.float64


ic| hod = regulari: tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
                              0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                            [ 0.0000e+00,  6.4765e+01,  0.0000e+00,  0.0000e+00,  2.1257e+00,
                              0.0000e+00, -1.8060e+01,  0.0000e+00,  0.0000e+00],
                            [ 0.0000e+00,  0.0000e+00,  8.2428e+01,  0.0000e+00,  1.5288e+00,
                              0.0000e+00,  0.0000e+00, ic-9.5970e+00,|  0.0000e+00 hod] ,
                            [= regulari : tensor(0.0000e+00[,[    0.0000,   0.0000e+00,  0.0000e+00,  2.3913e+020.0000,,     2.4439e+000.0000,   ,
                              0.0000e+000.0000,  ,0.0000e+00   ,  0.00000.0000e+00, ,-1.2743e+01   0.0000],,0.0000   
0.0000,
                               ,   0.0000],
                            [  0.0000,  63.7844,   0.0000,                               0.0000[ ,   1.9916,   0.0000, -18.2144,
                  

Exception occurred for parameters {'method': 'norm', 'lambda': 1e-08}: sigmoid(): argument 'input' (position 1) must be Tensor, not numpy.float64


0.2989[,,   0.0000e+00   0.0000e+00,  0.0000e+00,  0.0000e+00,0.0000,
                                0.0000e+00,   ,0.0000,
                               6.61890.0000e+00  ,,   0.0000],0.0000e+00  
,0.0000e+00                            [    0.0000e+000.0000, -,,  1.2743e+01   0.0000e+000.0000],
                            [,,    0.0000-, 2.3858e-01 ,-12.5634,  
-0.2390                              ,0.0000e+000.0000e+00,,  9.9880e+01  0.0000e+00,   ,    0.00000.0000e+00,0.0000e+00   0.0000,,    ,1.1504e+010.0000e+00ic|],
                                hod]  0.0000 ,1.8025e+00= regulari,  :12.1096]], ,dtype=torch

 tensor([[.                                                         dtypefloat64= 0.0000e+00)torch,
.0.0000e+00 ,-2.8979e+01float64)  0.0000e+00,  0.0000e+00,,  
  0.0000e+00,  0.0000e+00,  0.0000e+00,
                              0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
                            [ 0.0000e+00,  9.6497e+01,  0.0000e+00,  0.0000e+00,  1.9769e+00